### Inventory Management Problem

In [76]:
import numpy as np
import math
import dimod
from neal import SimulatedAnnealingSampler
from inventory import *

### Generating Sample Problem

In [77]:
#n_products = int(input("Number of Products:"))
#n_raw_materials = int(input("Number of Raw Materials:"))

n_products = 3
n_raw_materials = 7
print(f"Generating for {n_products} products and {n_raw_materials} raw materials.")

Generating for 3 products and 7 raw materials.


In [78]:
S = np.array([200., 240., 160.])
A = np.floor(10 * (np.random.random(size=(n_raw_materials, n_products)))) + 1
R = np.array([300., 360., 240., 270., 450., 600., 540.])  
C = np.floor(5 * np.random.random(size=n_raw_materials)) + 1  
D = np.array([30, 30, 30])

S.shape, A.shape, R.shape, C.shape, D.shape

((3,), (7, 3), (7,), (7,), (3,))

In [79]:
P = C @ A - S.T #objective matrix

### Encoding Functions

In [80]:
def bounded_coefficient_encoding(kappa_x, mu_x):
    """Bounded-Coefficient Encoding Algorithm"""
    #print(f"  Encoding variable with κ={kappa_x}, μ={mu_x}")
    
    if kappa_x < 2**(math.floor(math.log2(mu_x)) + 1):
        print(f"    Using binary encoding")
        log_kappa = math.floor(math.log2(kappa_x))
        binary_coeffs = [2**i for i in range(log_kappa + 1)]
        
        if sum(binary_coeffs[:-1]) < kappa_x:
            binary_coeffs[-1] = kappa_x - sum(binary_coeffs[:-1])
        
        print(f"    Coefficients: {binary_coeffs}")
        return binary_coeffs
    else:
        #print(f"    Using bounded-coefficient encoding")
        rho = math.floor(math.log2(mu_x)) + 1
        nu = kappa_x - sum(2**(i-1) for i in range(1, rho + 1))
        eta = math.floor(nu / mu_x)
        
        c_x = []
        for i in range(1, rho + 1):
            c_x.append(2**(i-1))
        for i in range(eta):
            c_x.append(mu_x)
        
        remainder = nu - eta * mu_x
        if remainder != 0:
            c_x.append(remainder)
        
        #print(f"    Coefficients: {c_x}")
        return c_x

def create_bounded_encoding_system(kappa_vec, mu_vec):
    """Create complete encoding system"""
    
    all_coefficients = []
    widths = []
    
    for i in range(len(kappa_vec)):
        coeffs = bounded_coefficient_encoding(kappa_vec[i], mu_vec[i])
        all_coefficients.append(coeffs)
        widths.append(len(coeffs))
    
    total_binary_vars = sum(widths)
    print(f"Total binary variables: {total_binary_vars}")
    
    return all_coefficients, widths, total_binary_vars

def create_bounded_encoding_matrix(n_variables, all_coefficients, widths):
    """Create encoding matrix C where x = C @ y"""
    total_binary_vars = sum(widths)
    C = np.zeros((n_variables, total_binary_vars))
    
    start_idx = 0
    for i in range(n_variables):
        end_idx = start_idx + widths[i]
        C[i, start_idx:end_idx] = all_coefficients[i]
        start_idx = end_idx
    
    return C

In [81]:
kappa_vec = D.astype(int)
mu_vec = [4, 4, 4]
all_coefficients, widths, total_binary_vars = create_bounded_encoding_system(kappa_vec, mu_vec)
C_enc = create_bounded_encoding_matrix(n_products, all_coefficients, widths)

Total binary variables: 27


### QUBO Formulation

In [ ]:
penalty_lambda = 1000

In [83]:
variable_names = []
for i in range(n_products):
    for j in range(widths[i]):
        variable_names.append(f"x{i+1}_bin{j+1}")

In [84]:
q_linear = P.T @ C_enc
q_linear.shape

(27,)

In [85]:
Q = np.zeros((total_binary_vars, total_binary_vars))
q_penalty = np.zeros(total_binary_vars)

In [86]:
for j in range(n_raw_materials):
    A_j = A[j, :].reshape(1, -1)
    Q_j = C_enc.T @ A_j.T @ A_j @ C_enc
    Q += penalty_lambda * Q_j

    q_penalty_j = -2 * penalty_lambda * R[j] * (A_j @ C_enc).flatten()
    q_penalty += q_penalty_j

In [87]:
q = q_linear + q_penalty

In [88]:
Q.shape, q.shape

((27, 27), (27,))

### BQM Formulation

In [89]:
bqm = dimod.BinaryQuadraticModel.empty('BINARY')

In [90]:
for i, var_name in enumerate(variable_names):
    bqm.add_variable(var_name, q[i])

for i in range(len(variable_names)):
    for j in range(i+1, len(variable_names)):
        if abs(Q[i, j]) > 1e-10:
            bqm.add_interaction(variable_names[i], variable_names[j], Q[i,j])

In [91]:
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm, num_reads=1000)

In [92]:
def decode_solution(solution, C_enc, variable_names, all_coefficients, widths):
    """Decode binary solution back to original variables"""
    
    y = np.array([solution[var_name] for var_name in variable_names])
    x = C_enc @ y
    
    print(f"Binary solution: {y}")
    print(f"Decoded solution x: {x}")
    
    return x, y

In [93]:
x_optimal, y_optimal = decode_solution(
    sampleset.first.sample, C_enc, variable_names, all_coefficients, widths
)

Binary solution: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Decoded solution x: [30. 30. 30.]
